# load the dataset MINIST

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
# Load data
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
# Normalize
train_images = train_images.astype("float32") / 255
test_images  = test_images.astype("float32") / 255

# Reshape
train_images = train_images.reshape((60000, 28 * 28))
test_images  = test_images.reshape((10000, 28 * 28))


In [ ]:
print(train_images.shape)
print(test_images.shape)

(60000, 784)
(10000, 784)


In [ ]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
print(train_labels.shape)
print(test_labels.shape)

(60000, 10)
(10000, 10)


# Build the network model

In [ ]:
from keras import models
from keras import layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
network.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

# Compile the network

In [ ]:
network.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

# Training dataset : try to "fit" the model

In [ ]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.8767 - loss: 0.4316
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.9652 - loss: 0.1205
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9789 - loss: 0.0716
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9846 - loss: 0.0509
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9891 - loss: 0.0374


# Test data set to evaluate

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9781 - loss: 0.0709


# Hyperparameter Tuning with Keras Tuner

In [ ]:
import keras_tuner as kt
import keras

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(784,)))

    # Number of hidden layers
    for i in range(hp.Int("num_layers", 2, 10)):
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation="relu"
            )
        )
        model.add(layers.Dropout(
            hp.Float(f"dropout_{i}", 0.2, 0.5, step=0.1)
        ))

    model.add(layers.Dense(10, activation="softmax"))

    learning_rate = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy", # Changed from sparse_categorical_crossentropy
        metrics=["accuracy"]
    )

    return model

In [ ]:
build_model(kt.HyperParameters())

<Sequential name=sequential_2, built=True>

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    directory="mnist_tuner",
    project_name="deep_mlp",
    overwrite=True # Added to ensure the latest model definition is used
)

In [ ]:
tuner.search(
    train_images, train_labels,
    validation_split=0.1,
    epochs=20,
    batch_size=128, # Changed from kt.Choice to a fixed integer
    callbacks=[
        keras.callbacks.EarlyStopping(patience=3, monitor="accuracy") # Corrected monitor from 'accuarcy' to 'accuracy'
    ]
)

Trial 20 Complete [00h 02m 55s]
val_accuracy: 0.9769999980926514

Best val_accuracy So Far: 0.9848333597183228
Total elapsed time: 00h 52m 40s


In [ ]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

print("Best hyperparameters found:")
for key, value in best_hps.values.items():
    print(f"  {key}: {value}")

print("\nSummary of the best model:")
best_model.summary()

Best hyperparameters found:
  num_layers: 2
  units_0: 160
  dropout_0: 0.4
  units_1: 480
  dropout_1: 0.30000000000000004
  learning_rate: 0.001
  units_2: 160
  dropout_2: 0.2
  units_3: 128
  dropout_3: 0.30000000000000004
  units_4: 224
  dropout_4: 0.30000000000000004
  units_5: 320
  dropout_5: 0.2
  units_6: 480
  dropout_6: 0.2
  units_7: 352
  dropout_7: 0.30000000000000004
  units_8: 64
  dropout_8: 0.30000000000000004
  units_9: 224
  dropout_9: 0.2

Summary of the best model:


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 160)            │       125,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 480)            │        77,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         4,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,690 (811.29 KB)

 Trainable params: 207,690 (811.29 KB)

 Non-trainable params: 0 (0.00 B)

# save the best model to use

In [ ]:
# Save the best model to an HDF5 file
best_model.save('best_model.h5')
print('Best model saved to best_model.h5')

Best model saved to best_model.h5


# find out the top 5 best trail

In [104]:
tuner.results_summary(num_trials=5)

Results summary
Results in mnist_tuner/deep_mlp
Showing 5 best trials
Objective(name="val_accuracy", direction="max")

Trial 15 summary
Hyperparameters:
num_layers: 2
units_0: 160
dropout_0: 0.4
units_1: 480
dropout_1: 0.30000000000000004
learning_rate: 0.001
units_2: 160
dropout_2: 0.2
units_3: 128
dropout_3: 0.30000000000000004
units_4: 224
dropout_4: 0.30000000000000004
units_5: 320
dropout_5: 0.2
units_6: 480
dropout_6: 0.2
units_7: 352
dropout_7: 0.30000000000000004
units_8: 64
dropout_8: 0.30000000000000004
units_9: 224
dropout_9: 0.2
Score: 0.9848333597183228

Trial 05 summary
Hyperparameters:
num_layers: 7
units_0: 448
dropout_0: 0.4
units_1: 416
dropout_1: 0.2
learning_rate: 0.001
units_2: 384
dropout_2: 0.30000000000000004
units_3: 224
dropout_3: 0.4
units_4: 96
dropout_4: 0.30000000000000004
units_5: 416
dropout_5: 0.4
units_6: 480
dropout_6: 0.2
units_7: 416
dropout_7: 0.30000000000000004
units_8: 96
dropout_8: 0.4
units_9: 320
dropout_9: 0.4
Score: 0.9838333129882812

Tria

# Load the best model

In [107]:
import tensorflow as tf
import time

# 1. Load the model from the local HDF5 file
loaded_model = tf.keras.models.load_model('best_model.h5')

# 2. Performance Evaluation (Timing the process)
start_time = time.time()
test_loss, test_accuracy = loaded_model.evaluate(test_images, test_labels, verbose=1)
end_time = time.time()

# 3. Output Results
print("\n" + "="*40)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"Total Inference Time: {end_time - start_time:.4f} seconds")
print(f"Average latency per image: {(end_time - start_time)/len(test_images):.6f} seconds")
print("="*40)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9775 - loss: 0.0826

Test Accuracy: 98.03%
Test Loss: 0.0714
Total Inference Time: 1.2170 seconds
Average latency per image: 0.000122 seconds
